# Tests d'extraction du texte des arrêtés


## Préliminaires

### Installation

```sh
mamba env create --file environment.yml
conda activate agperils-amp
```

### Fichier de test

In [1]:
from pathlib import Path

import pdftotext

In [2]:
# dossier contenant des PDF texte et image
RAW_PDF_DIR = Path("../data/raw/2022-03-08_export-actes/Export_@ctes_arretes_pdf")
# TODO détecter les conflits de noms entre fichiers dans les sous-dossiers de raw/ avant traitement et tri vers interim/
# dossier contenant les PDF image OCRisés
INT_PDF_DIR = Path("../data/interim/pdf")
INT_TXT_DIR = Path("../data/interim/txt")


In [3]:
INT_PDF_DIR.mkdir(exist_ok=True)
INT_TXT_DIR.mkdir(exist_ok=True)

def guess_pdf_type_and_extract_text(fp_pdf_in, fp_pdf_out, fp_txt_out):
    """Deviner le type de PDF (image ou texte) et extraire le texte.

    Si pdftotext renvoie du texte, le fichier est considéré PDF texte,
    sinon il est considéré PDF image et traité avec ocrmypdf.
    
    Parameters
    ----------
    fp_pdf_in: Path
        Chemin du fichier PDF à traiter.
    fp_pdf_out: Path
        Chemin du fichier PDF converti en PDF-A (avec OCR le cas échéant).
    fp_txt_out: Path
        Chemin du fichier txt contenant le texte extrait.
    """
    # 1. ouvrir le fichier PDF avec pdftotext
    with open(fp_pdf_in, "rb") as f:
        pdf = pdftotext.PDF(f)
    txt = "\n\n".join(pdf)
    # 2. si le texte n'est pas vide, alors c'est un PDF texte
    if txt.strip():
        # stocker le texte dans un fichier .txt
        print(f"PDF texte: {fp_pdf_in}")
        with open(fp_txt_out, "w") as f_txt:
            f_txt.write(txt)
        # convertir le PDF texte en PDF-A/OCR pour symétrie des traitements?
    else:
        # sinon c'est un PDF image
        # ex: RAW_PDF_DIR / "99_AR-013-211300025-20220128-A_2022_136-AR-1-1_1.pdf"
        print(f"PDF image: {fp_pdf_in}")
        # TODO utiliser ocrmypdf pour produire 2 fichiers: PDF-A/OCR + sidecar
        raise ValueError("GNE")


for fp_pdf_raw in RAW_PDF_DIR.glob("*.pdf"):
    fp_pdf_mod = INT_PDF_DIR / fp_pdf_raw.name
    fp_txt = INT_TXT_DIR / fp_pdf_raw.stem + ".txt"
    guess_pdf_type_and_extract_text(fp_pdf_raw, fp_pdf_mod, fp_txt)
# 
# TODO tester si (1) la sortie de pdftotext et (2) le sidecar (sur des PDF différents) sont globalement formés de façon similaire
# pour valider qu'on peut appliquer les mêmes regex/patterns d'extraction, ou s'il faut prévoir des variantes
# TODO stocker les fichiers txt (sidecar ou pdftotext) + métadonnées d'extraction (au moins colonnes, éventuellement table dédiée) incluant
# notamment ocrmypdf ou pdftotext (et les params?)
DOC_TEST_OCRD = "test-fra.pdf"
DOC_TEST_SIDE = "test-out-fra.txt"

../data/raw/2022-03-08_export-actes/Export_@ctes_arretes_pdf/99_AR-013-211300025-20220128-A_2022_136-AR-1-1_1.pdf


ValueError: GNE

In [4]:
# ocrmypdf, avec tesseract 4 (ubuntu)
# "-l fra" pour améliorer la reconnaissance de: "à", "è", "ê", apostrophe, "l'", "œ", "ô"
# TODO tester l'appel direct à ocrmypdf
# TODO tester "--clean" sur plusieurs PDF (aucun gain sur le pdf de test)
# TODO tester avec tesseract 5 (ubuntu PPA)
!ocrmypdf -l fra --sidecar {DOC_TEST_SIDE} {DOC_TEST} {DOC_TEST_OCRD}

Scanning contents: 100%|███████████████████████| 4/4 [00:00<00:00, 423.50page/s]
Start processing 4 pages concurrently
OCR: 100%|██████████████████████████████████| 4.0/4.0 [00:04<00:00,  1.15s/page]
Postprocessing...
PDF/A conversion: 100%|█████████████████████████| 4/4 [00:01<00:00,  3.98page/s]
Recompressing JPEGs: 0image [00:00, ?image/s]
Deflating JPEGs: 100%|████████████████████████| 4/4 [00:00<00:00, 225.22image/s]
JBIG2: 0item [00:00, ?item/s]
Optimize ratio: 1.39 savings: 28.2%
Output file is a PDF/A-2B (as expected)


In [8]:
# * pdftotext (xpdf/poppler) mélange le texte (comparé au fichier "sidecar" de ocrmypdf)
# * pdf2txt (pdfminer) mélange le texte (idem)
# * pdfplumber introduit des espaces et lignes superflus  

In [ ]:
# pdfplumber vs sidecar


## Tabula

* Dépôt GitHub: <https://github.com/chezou/tabula-py>
* Doc: <https://tabula-py.readthedocs.io>
* Autre doc: <https://aegis4048.github.io/parse-pdf-files-while-retaining-structure-with-tabula-py>

### Équivalent R
* tabulizer: <https://cran.r-hub.io/web/packages/tabulizer/vignettes/tabulizer.html>
* <https://stackoverflow.com/questions/42541849/extract-text-from-two-column-pdf-with-r>
* <http://blog.agileactors.com/blog/2017/9/5/how-to-extract-and-clean-data-from-pdf-files-in-r>
* <https://stackoverflow.com/questions/55770342/from-2-columns-of-text-to-1>



In [11]:
import tabula

In [12]:
dfs = tabula.read_pdf(DOC_TEST, pages='all', multiple_tables=True, guess=False, stream=True)

In [15]:
dfs[0]

,DEPARTEMENT DES,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,BOUCHES DU RHONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Allauch,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,un certain art de ville,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MAIRIE D‘ALLAUCH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Réf. : LF/ED/JR — 17/22 — 574034-574574,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFFICHE EN MAIRIE LE,78 JAN. 2022,NaN,NaN,NaN,NaN,NaN,NaN
6,"ARRETE N° 2022/ J 2 (,",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SEOXK _A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,ARRETE DE MISE EN SECURITE,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,PROCEDURE URGENTE,NaN,NaN,NaN,NaN,NaN


In [14]:
' '.join(str(x) for x in dfs[3].iloc[:, 0].values)

"logement ainsi Département. ARTICLE 7: Le Maire de la commune Administratif de Marseille dans Le tribunal administratif peut citoyens » accessible par ARTICLE 8: Monsieur Général des Services Commandant du agents de la Police l'exécution du présent arrété. nan nan nan nan nan"

## pdfalto

* Dépôt GitHub: <https://github.com/kermitt2/pdfalto>
* script pour extraire le texte: https://github.com/cneud/alto-ocr-text/blob/master/alto_ocr_text.py

À tester

## pdfplumber

Cf. travail Anthony sur "Demi ou moitié"

## Autres pistes

* <https://stackoverflow.com/questions/47533875/how-to-extract-table-as-text-from-the-pdf-using-python/47719296>
* LayoutParser
* <https://twitter.com/huggingface/status/1432717993637818383>


## Postface: extraire du texte depuis des fichiers PDF

* <https://stackoverflow.com/questions/22675690/if-identifying-text-structure-in-pdf-documents-is-so-difficult-how-do-pdf-reade>
* <https://www.bnf.fr/fr/techniques-et-formats-de-conversion-en-mode-texte>
